In [43]:
# Repro ready
import sys
sys.path.append('../')
import torch
from model_triton import t_scaled_dot_prod_attn_fwd3, t_scaled_dot_prod_attn_fwd3_t

#BS, H, N, D = 2, 2, 512, 64
BS, H, N, D = 2, 1, 16, 16
# qkv = torch.randn((BS, H, 3, N, D), device="cuda")
qkv = torch.randn((1, H, 3, N, D), device="cuda").expand(BS, H, 3, N, D)
mask = torch.tril(torch.ones((N,N), dtype=torch.bool, device="cuda")).unsqueeze(0).repeat(BS, 1, 1)
train=False #True
p_gen_aux = 42
N_RUNS = 1 #10

# print(mask.shape)
k=10
mask[1, k+1:, :]=0
# print(mask)

result, _ = t_scaled_dot_prod_attn_fwd3(qkv, mask, train, p_gen_aux)

result2, _ = t_scaled_dot_prod_attn_fwd3_t(qkv, mask, train, p_gen_aux)

assert torch.allclose(result, result2, rtol=5e-2, atol=1e-3), (result.shape, result2.shape, result[:2, :2, -4:, -4:], result2[:2, :2, -4:, -4:])

AssertionError: (torch.Size([2, 1, 16, 16]), torch.Size([2, 1, 16, 16]), tensor([[[[ 0.3227,  0.2860,  0.0628, -0.0717],
          [ 0.3847, -0.1799,  0.0733, -0.4351],
          [ 0.0875,  0.2422,  0.1104,  0.0177],
          [ 0.4990, -0.0590, -0.1283, -0.1561]]],


        [[[ 0.0242, -0.0036, -0.0193, -0.1388],
          [ 0.0242, -0.0036, -0.0193, -0.1388],
          [ 0.0242, -0.0036, -0.0193, -0.1388],
          [ 0.0242, -0.0036, -0.0193, -0.1388]]]], device='cuda:0'), tensor([[[[ 0.3227,  0.2858,  0.0628, -0.0717],
          [ 0.3845, -0.1799,  0.0733, -0.4351],
          [ 0.0873,  0.2421,  0.1104,  0.0178],
          [ 0.4991, -0.0590, -0.1284, -0.1562]]],


        [[[ 0.0060, -0.0009, -0.0048, -0.0347],
          [ 0.0060, -0.0009, -0.0048, -0.0347],
          [ 0.0060, -0.0009, -0.0048, -0.0347],
          [ 0.0060, -0.0009, -0.0048, -0.0347]]]], device='cuda:0'))